In [ ]:
from numpy import mean
from numpy import std
import pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from keras.models import Model, load_model
from keras.models import Sequential
from keras.layers import Conv2D, Input
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import BatchNormalization, Dropout,GaussianNoise
from keras.layers import Flatten
import tensorflow as tf
from keras.initializers import glorot_normal, RandomNormal, Zeros
import time
from keras import regularizers, optimizers
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping, LambdaCallback
import pandas as pd

## Convert labels to 260 categories using one hot code

In [ ]:
def dimension_change(label):
    digit = np.argmax(label[:10])
    alpha = np.argmax(label[-26:])
    new_index = digit*26+alpha
    new_label = np.zeros(260)
    new_label[new_index]=1
    return new_label

def label_change(x):
    dim = len(x)
    array = np.zeros([dim, 260])
    for i in range(len(x)):
        new_label = dimension_change(x[i])
#         np.concatenate((array, new_label), axis=0)
        array[i] = new_label
    return array

## Load data

In [ ]:
def load_data():
    with open('images_l.pkl', 'rb') as f:
        trainX = pickle.load(f)

    with open('labels_l.pkl', 'rb') as f:
        label = pickle.load(f)

    trainX = trainX.reshape((trainX.shape[0], 56, 56, 1))

    trainX= trainX.astype('float32')
    trainX = trainX / 255.0
    
    label = label_change(label)

    print(trainX.shape)
    print(label.shape)
    X_train, X_test, y_train, y_test = train_test_split(trainX, label, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

## Define model

In [ ]:
def define_model():
	model = Sequential()
	model.add(BatchNormalization(input_shape=(56, 56, 1)))
	model.add(GaussianNoise(0.1))
	model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
	model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))

	model.add(BatchNormalization())
	model.add(GaussianNoise(0.1))
	model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))

	model.add(BatchNormalization())
	model.add(GaussianNoise(0.1))
	model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.25))

	model.add(BatchNormalization())
	model.add(GaussianNoise(0.1))
	model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
	model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
	model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.25))
	model.add(GaussianNoise(0.05))

	model.add(BatchNormalization())
	model.add(Flatten())
	model.add(Dense(512, activation='relu'))
	model.add(Dropout(0.5))
	model.add(GaussianNoise(0.05))
	model.add(Dense(260, activation='softmax'))
	# compile model
	#opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
	return model

## Train the model and save the trained model

In [ ]:
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=10, verbose=1, factor=.75)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

with tf.device('/device:GPU:0'):
    start_time = time.time()
    X_train, X_test, y_train, y_test = load_data()
    model = define_model()
    print(model.summary())
    model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=128, epochs=120, callbacks=[lr_reduction, early_stopping])
    end_time = time.time()
    print("Trainig end! Time duration: " + str(int(end_time-start_time)) + " seconds")
    model.save("model1.h5")

## Do the training on unlabeled data

In [ ]:
loaded_model = tf.keras.models.load_model('model1.h5')

with open('images_ul.pkl', 'rb') as f:
    data_test = pickle.load(f)

data_test= data_test.reshape((data_test.shape[0], 56, 56, 1))
data_test= data_test.astype('float32')
data_test = data_test / 255.0

prediction = loaded_model.predict(data_test)
def toNormalLabel(prediction):
    string_list = []
    for i in range(len(prediction)):
        text = "000000000000000000000000000000000000"
        s=list(text)
        a = np.argmax(prediction[i])
        alpha = int(a % 26)
        digit = int((a-alpha) / 26)
        s[digit]= "1"
        s[10+alpha] = "1"
        st = "".join(s)
        string_list.append(st)
    return string_list
    
output = toNormalLabel(prediction)  
import pandas
df = pandas.DataFrame(data={"Category": output})
df["Category"] = df["Category"].apply(str) + '\t'
df
df.to_csv("./images_l2.csv",index=True)

## Load all data

In [ ]:
def load_data_all():
    with open('images_l.pkl', 'rb') as f:
        trainX = pickle.load(f)

    with open('labels_l.pkl', 'rb') as f:
        label = pickle.load(f)

    with open('image_ul.pkl', 'rb') as f:
        extra_trainX = pickle.load(f)

    extra_label = pd.read_csv('images_l2.csv')

    trainX = trainX + extra_trainX
    label = label + extra_label

    trainX = trainX.reshape((trainX.shape[0], 56, 56, 1))

    trainX= trainX.astype('float32')
    trainX = trainX / 255.0
    
    label = label_change(label)

    print(trainX.shape)
    print(label.shape)
    X_train, X_test, y_train, y_test = train_test_split(trainX, label, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

## Retrain the model using all data

In [ ]:
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=10, verbose=1, factor=.75)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

with tf.device('/device:GPU:0'):
    start_time = time.time()
    X_train, X_test, y_train, y_test = load_data_all()
    model = define_model()
    print(model.summary())
    model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=128, epochs=120, callbacks=[lr_reduction, early_stopping])
    end_time = time.time()
    print("Trainig end! Time duration: " + str(int(end_time-start_time)) + " seconds")
    model.save("model2.h5")

## Test and save result

In [ ]:
loaded_model = tf.keras.models.load_model('model2.h5')

with open('images_tets.pkl', 'rb') as f:
    data_test = pickle.load(f)

data_test= data_test.reshape((data_test.shape[0], 56, 56, 1))
data_test= data_test.astype('float32')
data_test = data_test / 255.0

prediction = loaded_model.predict(data_test)
def toNormalLabel(prediction):
    string_list = []
    for i in range(len(prediction)):
        text = "000000000000000000000000000000000000"
        s=list(text)
        a = np.argmax(prediction[i])
        alpha = int(a % 26)
        digit = int((a-alpha) / 26)
        s[digit]= "1"
        s[10+alpha] = "1"
        st = "".join(s)
        string_list.append(st)
    return string_list
    
output = toNormalLabel(prediction)  
import pandas
df = pandas.DataFrame(data={"Category": output})
df["Category"] = df["Category"].apply(str) + '\t'
df
df.to_csv("./submission.csv",index=True)